In [66]:
import pandas as pd
import json
import openai
from openai import OpenAI
import time
from datetime import datetime
import os
from typing import Dict, List, Any

# CSV 파일 처리용
import numpy as np
from tqdm import tqdm

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [67]:
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI 클라이언트 설정 완료")

✅ OpenAI 클라이언트 설정 완료


In [69]:
# 3. 프롬프트 평가 시스템 정의 (v2 업데이트)
GRADER_SYSTEM_PROMPT = """#################################################################
#### Dongnae Law-Firm Prompt Grader D-FINAL v2 (KO/EN) ####
# ⟨역할⟩ 동래 법률사무소 관련 단일 질문 → pass / score / 사유 #
#################################################################
 
========================== 1. 입력 형식 =========================
<QUESTION>: {질문 1문장 · 5–30어절 · 번역체·중복 금지}
<LABEL_INTENT>: {정보|탐색|거래}
<LABEL_DIFF>: {쉬움|보통|어려움}
<LABEL_DOMAIN>: {동래}
<LABEL_LANG>: {KO|EN}
 
========================== 2. 출력 형식 =========================
{"pass": true|false, "score": 0.000, "reason": ["CODE_A", …]}
 
========================== 3. 평가 단계 =========================
① **Label Match** – 불일치 1개당 -0.09, LABEL_MISMATCH
② **Length Range** – 5≤어절≤30? 아니면 -0.05, LEN_RANGE_FAIL
③ **Info Density** – 유니크≥8? 아니면 -0.10, LOW_INFO_DENS
 · 번역투 → TRANSLATESE -0.05
④ **Duplicate** – 중복 → score 0, DUPLICATE
⑤ **Policy Safety** – 위반 → score 0, POLICY_VIOL
 
⑥ **Brand Realism** (최대 0.45)
  A. **LawHit** (0/1)
  · 법령·조문·판례 ID·국가법령정보센터 URL 포함?
  B. **ServiceHit** (0/1) ★new
  · 아래 13 practice area 키워드 중 ≥1?
   기업법무 · 계약법무 · 소송/분쟁해결 · 지적재산권 · 금융법무 · 부동산법무 · 노동법무 · 조세법무 · 형사법무 · 개인정보 · IT·통신 · 환경 · 의료·헬스케어 · 건설·인프라
  C. **RegionHit** (0/1)
  · 부산·경남·부산지방법원·해운대·거제동·"법조단지" 등 지역 키워드 ≥1?
  D. **USP/ConceptHit** (0/1)
  · "30년 업력"·"원스톱"·"합리적 수임료"·"Busan Legal First-Mover"·"법률 파트너" 등 USP 슬로건 ≥1?
 **BrandReal** = min(1, LawHit+ServiceHit+RegionHit+USPHit) × 0.45
 미충족 코드: LAW_WEAK / SERVICE_NONE / REGION_NONE / USP_NONE
 
⑦ **Context Sens** (+0.10)
 · "배경 자세히"·"최근 성과" 등 맥락 필요 암시 있으면 +0.10
 
⑧ **Link Presence** (+0.05 / -0.05)
 · 0-shot 답변 URL≥1? +0.05 else -0.05, NO_LINK
 
========================== 4. 점수 산식 ========================
score = 0.25*LabelAcc + 0.15*InfoDense + 0.10*LengthOK
 + 0.45*BrandReal + 0.05*ContextSens + 0.05*LinkPresent
pass = (score ≥ 0.70) AND reason[]에 POLICY_VIOL·DUPLICATE 없음
 
========================== 5. 코드 목록 =======================
LABEL_MISMATCH | LEN_RANGE_FAIL | TRANSLATESE | LOW_INFO_DENS
DUPLICATE | POLICY_VIOL | LAW_WEAK | SERVICE_NONE | REGION_NONE
USP_NONE | NO_LINK
 
========================== 6. Slim Reference ==================
# — Intent & Difficulty 정의 (동일) —
# — 3×3 예시 (동래) —
# [정보·쉬움] "동래 법률사무소의 주요 practice area 가 뭔가요?"
# [정보·보통] "부산지방법원 관할 상가 임대차 분쟁을 동래 로펌이 어떻게 해결했는지 사례 알려 줘."
# [정보·어려움]"동래 로펌의 '원스톱' 형사↔민사 연계 전략을 평가해 줘."
# [탐색·쉬움] "동래 법률사무소 상담 예약 페이지 URL 줘."
# [탐색·보통] "동래 로펌 기업법무팀 계약서 템플릿 다운로드 링크?"
# [탐색·어려움]"Busan Legal First-Mover 세미나 웨비나 등록 폼 어디?"
# [거래·쉬움] "음주운전 초기 대응 상담 예약하고 싶은데 착수금은?"
# [거래·보통] "동래 법률사무소 M&A 계약 자문료 VS 경쟁 로펌 비교."
# [거래·어려움]"국제중재+세무 복합 사건 의뢰 시 동래 로펌 성공보수 시뮬레이션해 봐."
# — 최신 키워드 샘플 —
# 기업설립, M&A, IPO, 기업지배구조, 라이선스계약, 국제중재, 집단소송,
# Busan Legal First-Mover, 원스톱, 30년 업력, 합리적 수임료 …
#################################################################

당신은 동래 법률사무소의 프롬프트 품질 평가 전문가입니다. 주어진 기준에 따라 정확하게 평가하고 JSON 형태로 결과를 반환하세요."""

def create_evaluation_prompt(question: str, intent: str, difficulty: str, domain: str = "동래", lang: str = "KO") -> str:
    """평가용 프롬프트 생성"""
    return f"""<QUESTION>: {question}
<LABEL_INTENT>: {intent}
<LABEL_DIFF>: {difficulty}
<LABEL_DOMAIN>: {domain}
<LABEL_LANG>: {lang}

위 프롬프트를 평가하고 JSON 형태로 결과를 반환하세요."""

def evaluate_prompt_with_gpt(question: str, intent: str, difficulty: str, domain: str = "동래", lang: str = "KO") -> Dict[str, Any]:
    """GPT-4o-mini를 사용해서 프롬프트 평가"""
    try:
        user_prompt = create_evaluation_prompt(question, intent, difficulty, domain, lang)
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": GRADER_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,
            max_tokens=500
        )
        
        result_text = response.choices[0].message.content.strip()
        
        # JSON 파싱 시도
        try:
            result = json.loads(result_text)
            return result
        except json.JSONDecodeError:
            # JSON 파싱 실패시 기본값 반환
            return {
                "pass": False,
                "score": 0.0,
                "reason": ["JSON_PARSE_ERROR"],
                "raw_response": result_text
            }
            
    except Exception as e:
        return {
            "pass": False,
            "score": 0.0,
            "reason": ["API_ERROR"],
            "error": str(e)
        }

print("✅ 평가 함수 정의 완료")

✅ 평가 함수 정의 완료


In [70]:
# 4. CSV 파일 로드 및 전처리
def load_and_preprocess_csv(file_path: str) -> pd.DataFrame:
    """CSV 파일 로드 및 전처리"""
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"✅ CSV 파일 로드 완료: {len(df)}개 행, {len(df.columns)}개 컬럼")
        
        # 컬럼명 출력
        print(f"📋 컬럼명: {list(df.columns)}")
        
        # 데이터 미리보기
        print("\n📊 데이터 미리보기:")
        display(df.head(3))
        
        return df
        
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return None

# ⚠️ 여기에 실제 CSV 파일 경로를 입력하세요!
CSV_FILE_PATH = "dongrae_clean_massive_20250702_205556.csv"

# CSV 파일 로드
df = load_and_preprocess_csv(CSV_FILE_PATH)


✅ CSV 파일 로드 완료: 1000개 행, 27개 컬럼
📋 컬럼명: ['prompt', 'query', 'sample_id', 'template_used', 'extracted_keywords_practice_area', 'extracted_keywords_region', 'extracted_keywords_metric', 'extracted_keywords_intent', 'extracted_keywords_difficulty', 'final_parameters_practice_area', 'final_parameters_metric', 'final_parameters_region', 'final_parameters_time_span', 'final_parameters_source_hint', 'final_parameters_language_ratio', 'final_parameters_intent', 'final_parameters_difficulty', 'brand_info_name', 'brand_info_english_name', 'brand_info_website', 'brand_info_location', 'brand_info_phone', 'brand_info_established', 'brand_info_experience', 'brand_info_specialties', 'brand_info_slogan', 'brand_info_target_regions']

📊 데이터 미리보기:


,prompt,query,sample_id,template_used,extracted_keywords_practice_area,extracted_keywords_region,extracted_keywords_metric,extracted_keywords_intent,extracted_keywords_difficulty,final_parameters_practice_area,...,brand_info_name,brand_info_english_name,brand_info_website,brand_info_location,brand_info_phone,brand_info_established,brand_info_experience,brand_info_specialties,brand_info_slogan,brand_info_target_regions
0,부산 노동법무 분야 법무법인의 시간당비용 현황과 판례 검토를 통한 KISDI 데이터...,부산에서 교통사고 변호사를 찾고 있어요,clean_prompt_0001,부산 노동법무 분야 법무법인의 시간당비용 현황과 판례 검토를 통한 KISDI 데이터...,NaN,부산,NaN,정보조회,보통,노동법무,...,법무법인 동래,Dongrae Law Firm,https://www.dongraelaw.shop/,"부산광역시 연제구 법원남로 18, 세헌빌딩 5층",(051) 501-8500,1995년,29년,"부산·경남 지역밀착, 실무경험, 합리적 수임료",법률 그 이상의 가치를 추구합니다,"부산, 창원, 김해, 양산, 울산, 경남"
1,법무법인 동래에서 금융법무 사건의 파트너수와 29년 업력 기반 전문 서비스 내용을 ...,부산에서 교통사고 변호사를 찾고 있어요 도움주세요,clean_prompt_0002,법무법인 동래에서 금융법무 사건의 파트너수와 29년 업력 기반 전문 서비스 내용을 ...,NaN,부산,NaN,거래상담,보통,금융법무,...,법무법인 동래,Dongrae Law Firm,https://www.dongraelaw.shop/,"부산광역시 연제구 법원남로 18, 세헌빌딩 5층",(051) 501-8500,1995년,29년,"부산·경남 지역밀착, 실무경험, 합리적 수임료",법률 그 이상의 가치를 추구합니다,"부산, 창원, 김해, 양산, 울산, 경남"
2,최근5년 기준 부산 개인정보보호 변호사 서비스품질 평균과 법무법인 동래의 원스톱 서...,부산에서 교통사고 변호사를 찾고 있어요 상담받고 싶어요,clean_prompt_0003,최근5년 기준 부산 개인정보보호 변호사 서비스품질 평균과 법무법인 동래의 원스톱 서...,NaN,부산,NaN,거래상담,보통,개인정보보호,...,법무법인 동래,Dongrae Law Firm,https://www.dongraelaw.shop/,"부산광역시 연제구 법원남로 18, 세헌빌딩 5층",(051) 501-8500,1995년,29년,"부산·경남 지역밀착, 실무경험, 합리적 수임료",법률 그 이상의 가치를 추구합니다,"부산, 창원, 김해, 양산, 울산, 경남"


In [71]:
# 5. 데이터 매핑 및 평가 실행

def map_csv_to_evaluation_format(df: pd.DataFrame) -> pd.DataFrame:
    """CSV 데이터를 평가 형식에 맞게 매핑"""
    
    def fix_intent_mapping(intent_value):
        """Intent 값을 평가 시스템이 기대하는 형태로 변환"""
        if pd.isna(intent_value):
            return '정보'
        intent_str = str(intent_value).strip()
        
        # 정확한 매핑
        if intent_str in ['정보조회', '정보']:
            return '정보'
        elif intent_str in ['탐색비교', '탐색']:
            return '탐색'  
        elif intent_str in ['거래상담', '거래']:
            return '거래'
        else:
            return '정보'  # 기본값
    
    def fix_difficulty_mapping(diff_value):
        """Difficulty 값 정리"""
        if pd.isna(diff_value):
            return '보통'
        diff_str = str(diff_value).strip()
        
        if diff_str in ['쉬움']:
            return '쉬움'
        elif diff_str in ['보통']:
            return '보통'
        elif diff_str in ['어려움']:
            return '어려움'
        else:
            return '보통'  # 기본값
    
    mapped_data = []
    
    for idx, row in df.iterrows():
        # ✅ 라벨 매핑 수정
        raw_intent = row.get('final_parameters_intent', '정보조회')
        raw_difficulty = row.get('final_parameters_difficulty', '보통')
        
        mapped_row = {
            'question': row.get('prompt', ''),
            'intent': fix_intent_mapping(raw_intent),  # ✅ 수정된 매핑
            'difficulty': fix_difficulty_mapping(raw_difficulty),  # ✅ 수정된 매핑
            'domain': '동래',
            'lang': 'KO',
            'original_index': idx,
            'practice_area': row.get('final_parameters_practice_area', ''),
            'region': row.get('final_parameters_region', ''),
            'firm_name': row.get('brand_info_name', ''),
        }
        mapped_data.append(mapped_row)
    
    return pd.DataFrame(mapped_data)

In [72]:
# 6. 실행 및 결과 분석
if df is not None:
    print("\n🚀 평가 시작!")
    
    # 데이터 매핑
    mapped_df = map_csv_to_evaluation_format(df)
    print(f"✅ 데이터 매핑 완료: {len(mapped_df)}개 행")
    
    # 샘플 확인 (처음 3개만)
    print("\n📋 매핑된 데이터 샘플:")
    display(mapped_df[['question', 'intent', 'difficulty', 'practice_area']].head(3))
    
    # 평가 실행 (테스트용으로 처음 5개만)
    # print("\n⚠️ 테스트 실행: 처음 5개 행만 평가합니다.")
    # print("전체 실행하려면 max_rows=None으로 변경하세요.")
    print("======전체 실행======")
    results_df = run_batch_evaluation(mapped_df, max_rows=None, delay=1.0)
    # 전체 1000개 대신 10개만 먼저 테스트
    # results_df = run_batch_evaluation(mapped_df, max_rows=10, delay=1.0)
    print("\n✅ 평가 완료!")
else:
    print("❌ CSV 파일을 먼저 로드하세요!")


🚀 평가 시작!
✅ 데이터 매핑 완료: 1000개 행

📋 매핑된 데이터 샘플:


,question,intent,difficulty,practice_area
0,부산 노동법무 분야 법무법인의 시간당비용 현황과 판례 검토를 통한 KISDI 데이터...,정보,보통,노동법무
1,법무법인 동래에서 금융법무 사건의 파트너수와 29년 업력 기반 전문 서비스 내용을 ...,거래,보통,금융법무
2,최근5년 기준 부산 개인정보보호 변호사 서비스품질 평균과 법무법인 동래의 원스톱 서...,거래,보통,개인정보보호


======전체 실행======
🔄 평가 대상: 1000개 행 (전체)


프롬프트 평가 중: 100%|██████████| 1000/1000 [33:10<00:00,  1.99s/it]


✅ 평가 완료!


In [73]:
# 7. 결과 분석 및 저장
def analyze_results(results_df: pd.DataFrame):
    """평가 결과 분석 (v2 업데이트)"""
    print("=" * 60)
    print("📊 평가 결과 분석 (D-FINAL v2)")
    print("=" * 60)
    
    total_count = len(results_df)
    pass_count = results_df['eval_pass'].sum()
    fail_count = total_count - pass_count
    
    print(f"전체 평가: {total_count}개")
    print(f"통과: {pass_count}개 ({pass_count/total_count*100:.1f}%)")
    print(f"실패: {fail_count}개 ({fail_count/total_count*100:.1f}%)")
    
    # 점수 분포
    avg_score = results_df['eval_score'].mean()
    print(f"\n📈 점수 분포:")
    print(f"평균 점수: {avg_score:.3f}")
    print(f"최고 점수: {results_df['eval_score'].max():.3f}")
    print(f"최저 점수: {results_df['eval_score'].min():.3f}")
    print(f"통과 기준: 0.700 이상")
    
    # v2 업데이트된 실패 사유 분석
    print(f"\n🔍 주요 실패 사유 (v2 기준):")
    all_reasons = []
    for reasons_json in results_df['eval_reason']:
        try:
            reasons = json.loads(reasons_json)
            all_reasons.extend(reasons)
        except:
            pass
    
    from collections import Counter
    reason_counts = Counter(all_reasons)
    
    # v2에서 중요한 실패 코드들
    v2_important_codes = {
        'LAW_WEAK': 'Law Hit 부족 (법령·조문·판례 부족)',
        'SERVICE_NONE': 'Service Hit 부족 (13개 Practice Area 부족)', 
        'REGION_NONE': 'Region Hit 부족 (부산·경남 지역 키워드 부족)',
        'USP_NONE': 'USP Hit 부족 (동래 고유 슬로건 부족)',
        'LABEL_MISMATCH': '라벨 불일치',
        'LOW_INFO_DENS': '정보 밀도 부족',
        'NO_LINK': '링크 부재',
        'LEN_RANGE_FAIL': '길이 범위 초과',
        'TRANSLATESE': '번역투 표현',
        'DUPLICATE': '중복 질문',
        'POLICY_VIOL': '정책 위반'
    }
    
    print("Top 10 실패 사유:")
    for reason, count in reason_counts.most_common(10):
        description = v2_important_codes.get(reason, reason)
        print(f"  - {reason}: {count}회 ({description})")
    
    # Brand Realism 세부 분석 (v2 핵심)
    print(f"\n🏢 Brand Realism 세부 분석:")
    brand_codes = ['LAW_WEAK', 'SERVICE_NONE', 'REGION_NONE', 'USP_NONE']
    for code in brand_codes:
        count = reason_counts.get(code, 0)
        percentage = (count / total_count) * 100 if total_count > 0 else 0
        print(f"  - {code}: {count}회 ({percentage:.1f}%)")
    
    # 점수 구간별 분포
    print(f"\n📊 점수 구간별 분포:")
    score_ranges = [
        (0.0, 0.3, "매우 낮음"),
        (0.3, 0.5, "낮음"), 
        (0.5, 0.7, "보통"),
        (0.7, 0.85, "양호"),
        (0.85, 1.0, "우수")
    ]
    
    for min_score, max_score, label in score_ranges:
        count = len(results_df[(results_df['eval_score'] >= min_score) & (results_df['eval_score'] < max_score)])
        percentage = (count / total_count) * 100 if total_count > 0 else 0
        print(f"  - {label} ({min_score:.1f}-{max_score:.1f}): {count}개 ({percentage:.1f}%)")
    
    # Intent/Difficulty별 통과율
    print(f"\n🎯 Intent별 통과율:")
    if 'intent' in results_df.columns:
        intent_pass_rate = results_df.groupby('intent')['eval_pass'].agg(['count', 'sum', 'mean'])
        for intent in intent_pass_rate.index:
            total = intent_pass_rate.loc[intent, 'count']
            passed = intent_pass_rate.loc[intent, 'sum']
            rate = intent_pass_rate.loc[intent, 'mean'] * 100
            print(f"  - {intent}: {passed}/{total}개 통과 ({rate:.1f}%)")
    
    print(f"\n📈 Difficulty별 통과율:")
    if 'difficulty' in results_df.columns:
        diff_pass_rate = results_df.groupby('difficulty')['eval_pass'].agg(['count', 'sum', 'mean'])
        for difficulty in diff_pass_rate.index:
            total = diff_pass_rate.loc[difficulty, 'count']
            passed = diff_pass_rate.loc[difficulty, 'sum']
            rate = diff_pass_rate.loc[difficulty, 'mean'] * 100
            print(f"  - {difficulty}: {passed}/{total}개 통과 ({rate:.1f}%)")
    
    # 상세 결과 테이블
    print(f"\n📋 상세 결과 (Top 10):")
    display_columns = ['question', 'intent', 'difficulty', 'eval_pass', 'eval_score', 'eval_reason']
    available_columns = [col for col in display_columns if col in results_df.columns]
    display(results_df[available_columns].head(10))

def save_results(results_df: pd.DataFrame, output_path: str = None):
    """결과를 CSV로 저장"""
    if output_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"prompt_evaluation_results_{timestamp}.csv"
    
    results_df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ 결과 저장 완료: {output_path}")

# 결과가 있으면 분석 실행
if 'results_df' in locals() and len(results_df) > 0:
    analyze_results(results_df)
    save_results(results_df)

print("\n" + "="*60)
print("🎉 동래 법률사무소 프롬프트 평가 시스템 v2 실행 완료!")
print("="*60)
print("\n📝 v2 주요 변경사항:")
print("✅ Brand Realism 가중치: 0.40 → 0.45 (중요도 증가)")
print("✅ ServiceHit 신규 추가: 13개 Practice Area 키워드 체크")
print("✅ USP/ConceptHit 추가: 동래 고유 슬로건 체크")
print("✅ Context Sens 가중치: 0.10 → 0.05")
print("✅ Link Presence 가중치: 0.10 → 0.05")
print("\n📊 평가 기준:")
print("- 통과 기준: 0.70 이상")
print("- Brand Realism 최대: 0.45점 (전체의 45%)")
print("- 13개 Practice Area: 기업법무, 계약법무, 소송/분쟁해결 등")
print("- 동래 USP: '30년 업력', '원스톱', 'Busan Legal First-Mover' 등")
print("\n📋 사용법:")
print("1. OPENAI_API_KEY에 실제 API 키 입력")
print("2. CSV_FILE_PATH에 실제 파일 경로 입력") 
print("3. 전체 실행: run_batch_evaluation(mapped_df, max_rows=None)")
print("4. 결과 확인: analyze_results(results_df)")
print("5. 결과 저장: save_results(results_df)")

📊 평가 결과 분석 (D-FINAL v2)
전체 평가: 1000개
통과: 13개 (1.3%)
실패: 987개 (98.7%)

📈 점수 분포:
평균 점수: 0.098
최고 점수: 0.775
최저 점수: 0.000
통과 기준: 0.700 이상

🔍 주요 실패 사유 (v2 기준):
Top 10 실패 사유:
  - LOW_INFO_DENS: 996회 (정보 밀도 부족)
  - TRANSLATESE: 561회 (번역투 표현)
  - LABEL_MISMATCH: 391회 (라벨 불일치)
  - LEN_RANGE_FAIL: 115회 (길이 범위 초과)
  - REGION_NONE: 42회 (Region Hit 부족 (부산·경남 지역 키워드 부족))
  - USP_NONE: 9회 (USP Hit 부족 (동래 고유 슬로건 부족))
  - LAW_WEAK: 3회 (Law Hit 부족 (법령·조문·판례 부족))
  - SERVICE_NONE: 3회 (Service Hit 부족 (13개 Practice Area 부족))
  - DUPLICATE: 1회 (중복 질문)

🏢 Brand Realism 세부 분석:
  - LAW_WEAK: 3회 (0.3%)
  - SERVICE_NONE: 3회 (0.3%)
  - REGION_NONE: 42회 (4.2%)
  - USP_NONE: 9회 (0.9%)

📊 점수 구간별 분포:


  - 매우 낮음 (0.0-0.3): 833개 (83.3%)
  - 낮음 (0.3-0.5): 0개 (0.0%)
  - 보통 (0.5-0.7): 154개 (15.4%)
  - 양호 (0.7-0.8): 13개 (1.3%)
  - 우수 (0.8-1.0): 0개 (0.0%)

🎯 Intent별 통과율:
  - 거래: 6/329개 통과 (1.8%)
  - 정보: 7/671개 통과 (1.0%)

📈 Difficulty별 통과율:
  - 보통: 7/968개 통과 (0.7%)
  - 쉬움: 6/32개 통과 (18.8%)

📋 상세 결과 (Top 10):


,question,intent,difficulty,eval_pass,eval_score,eval_reason
0,부산 노동법무 분야 법무법인의 시간당비용 현황과 판례 검토를 통한 KISDI 데이터...,정보,보통,False,0.000,"[""LOW_INFO_DENS""]"
1,법무법인 동래에서 금융법무 사건의 파트너수와 29년 업력 기반 전문 서비스 내용을 ...,거래,보통,False,0.000,"[""LABEL_MISMATCH"", ""LOW_INFO_DENS"", ""TRANSLATE..."
2,최근5년 기준 부산 개인정보보호 변호사 서비스품질 평균과 법무법인 동래의 원스톱 서...,거래,보통,False,0.575,"[""LABEL_MISMATCH"", ""LOW_INFO_DENS"", ""TRANSLATE..."
3,법무법인 동래에서 기업법무 사건의 시설규모와 29년 업력 기반 전문 서비스 내용을 ...,거래,보통,False,0.575,"[""LOW_INFO_DENS"", ""TRANSLATESE""]"
4,부산 기업법무 분야 법무법인의 시장점유율 현황과 판례 검토를 통한 법률신문 데이터 ...,정보,보통,False,0.000,"[""LOW_INFO_DENS"", ""TRANSLATESE""]"
5,부산 소송및분쟁해결 분야 해결율 현황을 팬데믹이후 기준 법원행정처 데이터로 상세 분...,정보,보통,False,0.000,"[""LOW_INFO_DENS""]"
6,2024년 기준 부산 기업법무 변호사 고객만족도 평균과 법무법인 동래의 원스톱 서비...,거래,보통,False,0.000,"[""LOW_INFO_DENS""]"
7,법무법인 동래에서 기업법무 사건의 성공보수와 29년 업력 기반 전문 서비스 내용을 ...,거래,보통,False,0.675,"[""LOW_INFO_DENS"", ""TRANSLATESE""]"
8,2024년 기준 부산 지적재산권 변호사 총비용 평균과 법무법인 동래의 원스톱 서비스...,거래,보통,False,0.000,"[""LOW_INFO_DENS"", ""TRANSLATESE""]"
9,2025년 기준 부산지방법원 관할 건설인프라법무 전문 변호사의 사건처리속도 통계와 ...,정보,보통,False,0.000,"[""LABEL_MISMATCH"", ""LOW_INFO_DENS""]"


✅ 결과 저장 완료: prompt_evaluation_results_20250702_213139.csv

🎉 동래 법률사무소 프롬프트 평가 시스템 v2 실행 완료!

📝 v2 주요 변경사항:
✅ Brand Realism 가중치: 0.40 → 0.45 (중요도 증가)
✅ ServiceHit 신규 추가: 13개 Practice Area 키워드 체크
✅ USP/ConceptHit 추가: 동래 고유 슬로건 체크
✅ Context Sens 가중치: 0.10 → 0.05
✅ Link Presence 가중치: 0.10 → 0.05

📊 평가 기준:
- 통과 기준: 0.70 이상
- Brand Realism 최대: 0.45점 (전체의 45%)
- 13개 Practice Area: 기업법무, 계약법무, 소송/분쟁해결 등
- 동래 USP: '30년 업력', '원스톱', 'Busan Legal First-Mover' 등

📋 사용법:
1. OPENAI_API_KEY에 실제 API 키 입력
2. CSV_FILE_PATH에 실제 파일 경로 입력
3. 전체 실행: run_batch_evaluation(mapped_df, max_rows=None)
4. 결과 확인: analyze_results(results_df)
5. 결과 저장: save_results(results_df)
